# Step 1. Open the data file and read the general information

## Competition description
We need to build a predictive model that answers the question: “what sorts of people were more likely to survive?” using passenger data (ie name, age, gender, socio-economic class, etc).

## Import

In [123]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from pandas_profiling import ProfileReport
import sidetable

## Load data

In [124]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [125]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


## Step 2. Check the data

- Variable	Definition	Key
- survival	Survival	0 = No, 1 = Yes
- pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
- sex	Sex	
- Age	Age in years	
- sibsp	# of siblings / spouses aboard the Titanic	
- parch	# of parents / children aboard the Titanic	
- ticket	Ticket number	
- fare	Passenger fare	
- cabin	Cabin number	
- embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [127]:
df.stb.missing(style=True)

,missing,total,percent
Cabin,687,891,77.10%
Age,177,891,19.87%
Embarked,2,891,0.22%
PassengerId,0,891,0.00%
Survived,0,891,0.00%
Pclass,0,891,0.00%
Name,0,891,0.00%
Sex,0,891,0.00%
SibSp,0,891,0.00%
Parch,0,891,0.00%


In [128]:
df.describe(include='all')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Braund, Mr. Owen Harris",male,NaN,NaN,NaN,347082,NaN,B96 B98,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [129]:
df.sample(20)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
678,679,0,3,"Goodwin, Mrs. Frederick (Augusta Tyler)",female,43.0,1,6,CA 2144,46.9000,NaN,S
254,255,0,3,"Rosblom, Mrs. Viktor (Helena Wilhelmina)",female,41.0,0,2,370129,20.2125,NaN,S
65,66,1,3,"Moubarek, Master. Gerios",male,NaN,1,1,2661,15.2458,NaN,C
624,625,0,3,"Bowen, Mr. David John ""Dai""",male,21.0,0,0,54636,16.1000,NaN,S
215,216,1,1,"Newell, Miss. Madeleine",female,31.0,1,0,35273,113.2750,D36,C
697,698,1,3,"Mullens, Miss. Katherine ""Katie""",female,NaN,0,0,35852,7.7333,NaN,Q
672,673,0,2,"Mitchell, Mr. Henry Michael",male,70.0,0,0,C.A. 24580,10.5000,NaN,S
266,267,0,3,"Panula, Mr. Ernesti Arvid",male,16.0,4,1,3101295,39.6875,NaN,S
534,535,0,3,"Cacic, Miss. Marija",female,30.0,0,0,315084,8.6625,NaN,S
508,509,0,3,"Olsen, Mr. Henry Margido",male,28.0,0,0,C 4001,22.5250,NaN,S


# Split the source data into a training set and validation set

In [130]:
df_train, df_valid = train_test_split(df, test_size=0.25, random_state=12345)

# Investigate the quality of different models by changing hyperparameters. Briefly describe the findings of the study.

We are asked if the person will survive or not. Then any wrong recommendation will be considered as error. Therefore we will evaluate the model with accuracy metric.

This is a classification task so we will check which learning algorithm for classification yields the best accuracy. The models we will check are: Decision tree, Random forest and Logistic regression

In [131]:
target_train = df_train['Survived']
target_valid = df_valid['Survived']

features = ["Pclass", "Sex", "SibSp", "Parch", 'Fare']
features_train = pd.get_dummies(df_train[features])
features_valid = pd.get_dummies(df_valid[features])
features_test = pd.get_dummies(df_test[features])


In [132]:
features_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Pclass      418 non-null    int64  
 1   SibSp       418 non-null    int64  
 2   Parch       418 non-null    int64  
 3   Fare        417 non-null    float64
 4   Sex_female  418 non-null    uint8  
 5   Sex_male    418 non-null    uint8  
dtypes: float64(1), int64(3), uint8(2)
memory usage: 14.0 KB


In [133]:
features_test

,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male
0,3,0,0,7.8292,0,1
1,3,1,0,7.0000,1,0
2,2,0,0,9.6875,0,1
3,3,0,0,8.6625,0,1
4,3,1,1,12.2875,1,0
...,...,...,...,...,...,...
413,3,0,0,8.0500,0,1
414,1,0,0,108.9000,1,0
415,3,0,0,7.2500,0,1
416,3,0,0,8.0500,0,1


In [134]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [135]:
features_test.loc[features_test['Fare'].isna(), :]

,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male
152,3,0,0,NaN,0,1


In [136]:
features_test.dropna(inplace=True)
features_test.loc[features_test['Fare'].isna(), :]

,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male


In [137]:
df_test.loc[df_test['Fare'].isna(), :]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [138]:
# df_test.dropna(inplace=True)
df_test = df_test[df_test['Fare'].notna()]
df_test.loc[df_test['Fare'].isna(), :]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [139]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


## Decision tree

loop tree depth to optimize the model depth with best accuracy

In [140]:
best_depth = 0
best_result = 0
for depth in range(1,20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # create a model with the given depth
    model.fit(features_train, target_train) # train the model
    predictions_valid = model.predict(features_valid) # get the model's predictions
    result = accuracy_score(target_valid, predictions_valid) # calculate the accuracy
    if result > best_result:
        best_result = result
        best_depth = depth
    print("max_depth =", depth, ": ", end='')
    print(result)
print('')
print('')
print("best_depth =", best_depth, ": ", end='')
print(best_result)

max_depth = 1 : 0.7802690582959642
max_depth = 2 : 0.7309417040358744
max_depth = 3 : 0.7847533632286996
max_depth = 4 : 0.7847533632286996
max_depth = 5 : 0.7668161434977578
max_depth = 6 : 0.7757847533632287
max_depth = 7 : 0.7847533632286996
max_depth = 8 : 0.7982062780269058
max_depth = 9 : 0.7892376681614349
max_depth = 10 : 0.7937219730941704
max_depth = 11 : 0.7847533632286996
max_depth = 12 : 0.7713004484304933
max_depth = 13 : 0.7802690582959642
max_depth = 14 : 0.7847533632286996
max_depth = 15 : 0.7847533632286996
max_depth = 16 : 0.7757847533632287
max_depth = 17 : 0.7892376681614349
max_depth = 18 : 0.7892376681614349
max_depth = 19 : 0.7892376681614349


best_depth = 8 : 0.7982062780269058


Depth 8 give the highest accuracy - 0.798

## Random forest

loop tree number of trees to optimize the model with best accuracy

In [141]:
best_score = 0
best_est = 0
for est in range(1, 10): # choose hyperparameter range
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # set number of trees
    model.fit(features_train, target_train) # train model on training set
    score = model.score(features_valid, target_valid) # calculate accuracy score on validation set
    if score > best_score:
        best_score = score# save best accuracy score on validation set
        best_est = est# save number of estimators corresponding to best accuracy score

print("Accuracy of the best model on the validation set (n_estimators = {}): {}".format(best_est, best_score))

Accuracy of the best model on the validation set (n_estimators = 6): 0.7802690582959642


## Logistic regression

In [142]:
model = LogisticRegression(random_state=12345, solver='lbfgs')
model.fit(features_train, target_train) 
score = model.score(features_valid, target_valid) 
print('The accuracy of the validation set is:', score)

The accuracy of the validation set is: 0.7802690582959642


## Model study conclusion
The highest accuracy we got is 0.79 and the model that allowed it is decision tree with depth of 10. Other models with different hyperparameters couldn't compete that. Therefore we select this model with and will check the quality of this model with the test set

# Train the winning model with all data

Break point here to select best model

break

Depth 8 give the highest accuracy - 0.798 with DecisionTreeClassifier

In [144]:
target = df['Survived']

features = ["Pclass", "Sex", "SibSp", "Parch", 'Fare']
features = pd.get_dummies(df[features])

model = DecisionTreeClassifier(random_state=12345, max_depth=8)
model.fit(features, target) # fit with train data
predictions_test = model.predict(features_test) # predict with test data

In [145]:
output = pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Survived': predictions_test})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [ ]:
#profile = ProfileReport(df)
#profile